# Making a movie of reaction-diffusion concentrations

We recommend creating and using a virtual environment for NetPyNE tutorials.  To do so, enter the following commands into your terminal:

    mkdir netpyne_tuts
    cd netpyne_tuts
    python3 -m venv env
    source env/bin/activate
    python3 -m pip install --upgrade pip setuptools wheel 
    python3 -m pip install --upgrade ipython ipykernel jupyter
    python3 -m pip install --upgrade neuron
    git clone https://github.com/Neurosim-lab/netpyne.git
    python3 -m pip install -e netpyne
    ipython kernel install --user --name=env
    
For this tutorial, you will also need to install `natsort` and `imageio`.

    python3 -m pip install natsort imageio

Then you can copy the example directory we will be using into `netpyne_tuts`, copy this notebook tutorial into it, and compile the mod files.
    
    cp -r netpyne/examples/rxd_net .
    cp netpyne/netpyne/tutorials/rxd_movie_tut/rxd_movie_tut.ipynb rxd_net
    cd rxd_net
    nrnivmodl mod
    
Finally, you can launch this tutorial in a Jupyter notebook.

    jupyter notebook rxd_movie_tut.ipynb

Note that the network parameters are defined in `netParams.py`, the simulation configuration is specified in `cfg.py` and the steps to actually run the simulation are in `init.py`.

From the terminal, you could run this simulation with the command `python3 init.py`.  Or, if you have MPI properly installed, you could run the sim on four cores with the command `mpiexec -np 4 nrniv -python -mpi init.py`.

To run the simulation from this notebook, you would execute `%run init.py` or `!mpiexec -np 4 nrniv -python -mpi init.py`.

However, we need to modify the simulation run so that a movie frame (figure) is generated at specified times.  You can modify `init.py` to do this, but here we will do it interactively.

First, lets look at what's in `init.py`:

    from netpyne import sim
    from netParams import netParams
    from cfg import cfg

    sim.initialize(netParams, cfg)
    sim.net.createPops()
    sim.net.createCells()
    sim.net.connectCells()
    sim.net.addStims()
    sim.net.addRxD()
    sim.setupRecording()
    sim.simulate()
    sim.analyze()

We want to replace `sim.simulate()` with `sim.runSimWithIntervalFunc()`, which pauses at a set interval and executes the specified function.  See more details on `runSimWithIntervalFunc` here: http://netpyne.org/netpyne.sim.run.html#netpyne.sim.run.runSimWithIntervalFunc.

The function `runSimWithIntervalFunc` requires two arguments: the time interval at which to execute the function (`interval`) and the function to be executed (`func`).  It also has two optional arguments: a limited time range over which to execute the function (`timeRange`) and a dictionary of arguments to feed into the function to be executed (`funcArgs`).

For this example, which runs for 1000 ms, we will make a short movie with 10 frames by setting `interval=100`.  We will use the function `sim.analysis.plotRxDConcentration` and we want to plot the calcium concentration in the extracellular space.  We also need to set `saveFig` to `'movie'`, and set the colorbar limits (so they stay the same in each movie frame).  In order to feed these arguments into the plotting function at each time step, we will create a dictionary:

In [ ]:
plotArgs = {
    'speciesLabel': 'ca',
    'regionLabel' : 'ecs', 
    'saveFig'     : 'movie',
    'showFig'     : False,
    'clim'        : [1.9997, 2.000],
}

At this point, we can replace `sim.simulate()` in our `init.py` file with `sim.runSimWithIntervalFunc(100.0, sim.analysis.plotRxDConcentration, timeRange=None, funcArgs=plotArgs)`.

Then we can run the simulation.

In [ ]:
from netpyne import sim
from netParams import netParams
from cfg import cfg

sim.initialize(netParams, cfg)
sim.net.createPops()
sim.net.createCells()
sim.net.connectCells()
sim.net.addStims()
sim.net.addRxD()
sim.setupRecording()
#sim.simulate()
sim.runSimWithIntervalFunc(100.0, sim.analysis.plotRxDConcentration, timeRange=None, funcArgs=plotArgs)
sim.analyze()

This should run the simulation, pausing every 100 ms to create a reaction-diffusion concentration plot.

At this point, we can create a movie (an animated gif) from our frames.

In [ ]:
import os
import natsort
import imageio

images = []
filenames = natsort.natsorted([file for file in os.listdir() if 'movie' in file and file.endswith('.png')])
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('rxd_conc_movie.gif', images)

Your movie should show up below (you may have to edit and then re-execute the following cell first):

<img src="rxd_conc_movie.gif">

Congratulations!  You have performed a reaction-diffusion simulation and created a movie of an ionic concentration!

As an exercise, to generate a movie of membrane voltages in the cells of the network, you can create a dictionary of the desired arguments for `plotShape`:

    shapePlotArgs = {
        'includePre' : ['all'], 
        'includePost': ['all'], 
        'cvar'       : 'voltage', 
        'clim'       : [-70, -20], 
        'saveFig'    : 'movie', 
        'showFig'    : False,
    }

And then replace `sim.simulate()` with:

    sim.runSimWithIntervalFunc(10.0, sim.analysis.plotShape, timeRange=[0, 200], funcArgs=shapePlotArgs)